<a href = 'https://dhds.cdc.gov/CR?ReportType=Years&LocationId=59&StratCatId=RACE&YearId=YR5&CompareColumns=YR5&ShowCategoryIds=PREVENT,BARRIER&SearchString=&ShowReportOptions=false&CategoryFilterId=&StratFilterId=&ShowCI=true&ShowWN=true&HideMissingDataRows=false'>Data Source</a>

In [1]:
import pandas as pd
import numpy as np 
import os
import altair as alt

In [432]:
pd.options.mode.chained_assignment = None

In [424]:
df = pd.read_csv("../data/preventitive_care.csv")

<h2>Manipulate DataFrame</h2>

In [425]:
df = df.drop(columns = ['FootnoteText', 'Url', 'FootnoteType'])

In [426]:
preventitive_care = df[(df.Category == 'Prevention & Screenings') & (df.Year == 2020.0)
                       & (df.Response == 'Yes')]

In [427]:
preventitive_care['Data_Value'] = preventitive_care.Data_Value / 100
preventitive_care['High_Confidence_Limit'] = preventitive_care.High_Confidence_Limit / 100
preventitive_care['Low_Confidence_Limit'] = preventitive_care.Low_Confidence_Limit / 100

/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_18258/2997796587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preventitive_care['Data_Value'] = preventitive_care.Data_Value / 100
/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_18258/2997796587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preventitive_care['High_Confidence_Limit'] = preventitive_care.High_Confidence_Limit / 100
/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_18258/2997796587.py:3: SettingWithCop

In [7]:
disabled_mean = pd.DataFrame(preventitive_care[preventitive_care.Stratification1 == 'Any Disability']\
                             .groupby('Question')['Data_Value'].mean())

disabled_mean = disabled_mean.reset_index()

In [8]:
vals = ['Data_Value', 'High_Confidence_Limit', 'Low_Confidence_Limit']
mean_by_type = preventitive_care[preventitive_care.Stratification1 != 'Any Disability']\
                .groupby(['Stratification1', 'Question'])[vals].mean()
mean_by_type = mean_by_type.reset_index()

In [9]:
mean_by_type = mean_by_type.merge(disabled_mean, on = 'Question', suffixes = ('', '_Disabled_Mean'))
mean_by_type['delta'] = mean_by_type['Data_Value'] - mean_by_type['Data_Value_Disabled_Mean']

mean_by_type['upper_val_delta'] = mean_by_type['High_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']
mean_by_type['lower_val_delta'] = mean_by_type['Low_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']

#mean_by_type['delta'] = (mean_by_type['Data_Value'] - mean_by_type['Data_Value_Disabled_Mean']) / mean_by_type['Data_Value_Disabled_Mean']

#mean_by_type['upper_val_delta'] = (mean_by_type['High_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']) / mean_by_type['Data_Value_Disabled_Mean']
#mean_by_type['lower_val_delta'] = (mean_by_type['Low_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']) /  mean_by_type['Data_Value_Disabled_Mean']

In [10]:
 mean_by_type['radio_option'] = mean_by_type.Stratification1.apply(lambda s: np.where(s == 'No Disability', 
                                                                                      'Include Non-Disabled People', 
                                                                                      'Exclude Non-Disabled People'))

In [11]:
mean_by_type['margin_of_error'] = mean_by_type['High_Confidence_Limit'] - mean_by_type['Data_Value']

Generate lower and upper bounds for scale parameter

In [17]:
mean_by_type['annotation_indicator'] = mean_by_type['Data_Value_Disabled_Mean'] - mean_by_type['Data_Value_Disabled_Mean']

In [18]:
mean_by_type['disabled_binary'] = mean_by_type['Stratification1'].apply(lambda s: np.where(s == 'No Disability', 
                                                                                           'No Disability', 
                                                                                           'Disabled'))

In [19]:
questions = list(mean_by_type.Question.unique())
zero_df = pd.DataFrame({'x': [0]})
sorting = ['Cognitive Disability', 'Hearing Disability', 'Mobility Disability', 
           'Vision Disability', 'No Disability']
radio_options = ['Include Non-Disabled People', 'Exclude Non-Disabled People']

select_care_type = alt.selection_single(
    fields = ['Question'], 
    init = {'Question' : questions[0]}, 
    bind = alt.binding_select(options = questions, name = 'Select Preventive Care Type: '), 
    on = 'dragover', 
    clear = 'false'
)

# include_non_disabled_radio = alt.binding_radio(options = radio_options)
# non_disabled_select = alt.selection_single(fields = ['radio_option'], 
#                                            bind = include_non_disabled_radio, 
#                                            name = "Rating")

input_checkbox = alt.binding_checkbox()
checkbox_selection = alt.selection_single(bind= input_checkbox, name = "Ability Types")

# size_checkbox_condition = alt.condition(checkbox_selection,
#                                         alt.SizeValue(25),
#                                         alt.Size('Hundred_Million_Production:Q')
#                                        )


lines = alt.Chart(mean_by_type).mark_rule(
    color = 'blue', 
    size = 3
).encode(
    alt.Y('Stratification1:N', 
          title = 'Type of Disability', sort = sorting), 
    alt.X('upper_val_delta:Q', 
          title = 'Difference from Average For All Disabilities (Percentage Points)',
          scale = alt.Scale(domainMid = 0)),
    alt.X2('lower_val_delta:Q'),
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')
               #alt.Tooltip('High_Confidence_Limit:Q', title = 'High Estimate', format = '.2%'), 
               #alt.Tooltip('Low_Confidence_Limit:Q', title = 'Low Estimate', format = '.2%')
              ]
).transform_filter(select_care_type)

mean = alt.Chart(mean_by_type).add_selection(select_care_type).mark_point(
    size = 50
).encode(
    y = alt.Y('Stratification1:N', sort = sorting), 
    x = alt.X('delta:Q', 
              scale = alt.Scale(domainMid = 0)), 
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')
               #alt.Tooltip('High_Confidence_Limit:Q', title = 'High Estimate', format = '.2%'), 
               #alt.Tooltip('Low_Confidence_Limit:Q', title = 'Low Estimate', format = '.2%')
              ]
).transform_filter(select_care_type)

zero = alt.Chart(zero_df).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('x')
)

mean_annotation_num = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 135, 
    dy = 140, 
    align = 'left'
).encode(
    x = alt.X('annotation_indicator:Q'), 
    text = alt.Text('Data_Value_Disabled_Mean', format = '.0%')
).transform_filter(select_care_type)

mean_annotation_text = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 5, 
    dy = 140, 
    align = 'left',
    text = 'Average For Disabled People:'
).encode(
    x = alt.X('annotation_indicator:Q')
)

alt.layer(zero, lines, mean, mean_annotation_num, mean_annotation_text
).properties(
    width = 600, 
    height = 300
)

alt.LayerChart(...)

With Raw Values

In [20]:
questions = list(mean_by_type.Question.unique())
sorting = ['Cognitive Disability', 'Hearing Disability', 'Mobility Disability', 
           'Vision Disability', 'No Disability']

title = alt.TitleParams("Many Disabled People Aren't Up To Date on Important Preventitive Care", 
                        anchor = 'start', fontSize = 18, dx = 105)

select_care_type = alt.selection_single(
    fields = ['Question'], 
    init = {'Question' : questions[0]}, 
    bind = alt.binding_select(options = questions, name = 'Select Preventive Care Type: '), 
    on = 'dragover', 
    clear = 'false'
)

lines = alt.Chart(mean_by_type, title = title).mark_rule(
    color = 'blue', 
    size = 3
).encode(
    alt.Y('Stratification1:N', sort = sorting,
          axis = alt.Axis(title = 'Type of Disability', titleFontSize = 13, labelFontSize = 10.5)
         ), 
    alt.X('High_Confidence_Limit:Q', 
          axis = alt.Axis(title = 'Percent Up To Date', titleFontSize = 13, labelFontSize = 11, format = '.0%')
          , 
          scale = alt.Scale(domain = [0.35, 0.9], zero = False)
         ),
    alt.X2('Low_Confidence_Limit:Q'),
    color = alt.Color('disabled_binary:N', legend = alt.Legend(title = 'Disability Status', 
                                                              titleFontSize = 15, labelFontSize = 12)
                     ),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
).transform_filter(select_care_type)


mean = alt.Chart(mean_by_type).add_selection(select_care_type).mark_point(
    size = 50
).encode(
    y = alt.Y('Stratification1:N', sort = sorting), 
    x = alt.X('Data_Value:Q'
              #, scale = alt.Scale(range = [0.35, 1.00])
             ), 
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
).transform_filter(select_care_type)


zero = alt.Chart(mean_by_type).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('Data_Value_Disabled_Mean'),
    tooltip = alt.Tooltip('Data_Value_Disabled_Mean:Q', title = 'Average For Disabled People', format = '.0%')
).transform_filter(select_care_type)

mean_annotation_num = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 135, 
    dy = 140, 
    align = 'left'
).encode(
    x = alt.X('Data_Value_Disabled_Mean:Q'), 
    text = alt.Text('Data_Value_Disabled_Mean', format = '.0%')
).transform_filter(select_care_type)

mean_annotation_text = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 5, 
    dy = 140, 
    align = 'left',
    text = 'Average For Disabled People:'
).encode(
    x = alt.X('Data_Value_Disabled_Mean:Q')
).transform_filter(select_care_type)

c = alt.layer(zero, lines, mean, mean_annotation_num, mean_annotation_text
).properties(
    width = 700, 
    height = 400
)
c.interactive()

alt.LayerChart(...)

<h2>Version with Small Multiples</h2>

In [186]:
selection_1 = alt.selection_single(encodings = ['color'], on = 'mouseover')
color_condition = alt.condition(selection_1, alt.Color("Stratification1:N"), alt.value('gray'))


lines = alt.Chart().mark_rule(
    #color = 'blue', 
    size = 3
).encode(
    alt.Y('Stratification1:N', sort = sorting,
          axis = alt.Axis(title = 'Type of Disability', titleFontSize = 13, labelFontSize = 10.5)
         ), 
    alt.X('High_Confidence_Limit:Q', 
          axis = alt.Axis(title = 'Percent Up To Date', titleFontSize = 13, labelFontSize = 11, format = '.0%'), 
          scale = alt.Scale(domain = [0.35, 0.85], zero = False)),
    alt.X2('Low_Confidence_Limit:Q'),
    color = alt.Color('Stratification1:N', legend = alt.Legend(title = 'Disability Status', titleFontSize = 15, labelFontSize = 12)),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
)

lines = lines.encode(color = color_condition)

mean = alt.Chart().mark_point(
    filled = True,
    size = 50
).encode(
    y = alt.Y('Stratification1:N', sort = sorting), 
    x = alt.X('Data_Value:Q'), 
    color = alt.Color('Stratification1:N')
)

mean = b.add_selection(selection_1).encode(color = color_condition)


zero = alt.Chart().mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('Data_Value_Disabled_Mean'),
    tooltip = alt.Tooltip('Data_Value_Disabled_Mean:Q', title = 'Average For Disabled People', format = '.0%')
)

mean_annotation_text = alt.Chart().mark_text(
    fontSize = 9.5, 
    dx = 5, 
    dy = 10, 
    align = 'left',
    text = 'Average For Disabled People'
).encode(
    x = alt.X('Data_Value_Disabled_Mean:Q')
)

l = alt.layer(zero, mean_annotation_text, lines, mean, 
          data = mean_by_type
).facet('Question', 
        columns = 2, 
        title = title
)

l.spec.width = 450
l.spec.height = 200
l

alt.FacetChart(...)

<h2>Fake Facet With Custom Scales</h2>

In [313]:

lines = alt.Chart(mean_by_type).mark_rule(
    #color = 'blue', 
    size = 3
).encode(
    alt.Y('Stratification1:N', sort = sorting,
          axis = alt.Axis(title = 'Type of Disability', titleFontSize = 13, labelFontSize = 10.5)
         ), 
    alt.X('High_Confidence_Limit:Q'),
    alt.X2('Low_Confidence_Limit:Q'),
    color = alt.Color('Stratification1:N', legend = alt.Legend(title = 'Disability Status', titleFontSize = 15, labelFontSize = 12)),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
)

#lines = lines.encode(color = color_condition)

mean = alt.Chart(mean_by_type).mark_point(
    filled = True,
    size = 50
).encode(
    y = alt.Y('Stratification1:N', sort = sorting), 
    x = alt.X('Data_Value:Q'), 
    color = alt.Color('Stratification1:N')
)

#mean = b.add_selection(selection_1).encode(color = color_condition)


zero = alt.Chart(mean_by_type).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('Data_Value_Disabled_Mean'),
    tooltip = alt.Tooltip('Data_Value_Disabled_Mean:Q', title = 'Average For Disabled People', format = '.0%')
)

mean_annotation_text = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 9, 
    dy = 10, 
    align = 'left',
    text = 'Average For Disabled People'
).encode(
    x = alt.X('Data_Value_Disabled_Mean:Q')
)


In [321]:
questions = list(mean_by_type.Question.unique())
sub_charts_components = []

for question in questions:
    sub_chart_mean = mean.transform_filter(alt.datum.Question == question)
    sub_chart_line = lines.transform_filter(alt.datum.Question == question)
    sub_chart_zero = zero.transform_filter(alt.datum.Question == question)
    sub_chart_annotation = mean_annotation_text.transform_filter(alt.datum.Question == question)
    sub_charts_components.append((question, sub_chart_mean, sub_chart_line, sub_chart_zero, sub_chart_annotation))

In [325]:
selection_1 = alt.selection_single(encodings = ['color'], on = 'mouseover')
color_condition = alt.condition(selection_1, alt.Color("Stratification1:N"), alt.value('gray'))

domains = [[0.35, 0.55], [0.6, 0.9], [0.6, 0.9], [0.7, 0.9], [0.6, 0.9], [0.4, 0.8]]
sub_charts = []

for i in range(len(sub_charts_components)):

    q, m, l, z, a = sub_charts_components[i]
    if (i == 4) or (i == 5):
        title = 'Percent Up To Date'
    else:
        title = ''
    domain = domains[i]
    l = l.add_selection(selection_1)
    l = l.encode(x = alt.X('High_Confidence_Limit:Q', scale = alt.Scale(domain = domain, zero = False), axis = alt.Axis(title = title, titleFontSize = 13, labelFontSize = 11, format = '.0%')), 
                 color = color_condition)
    m = m.encode(color = color_condition)
    sub_chart = alt.layer(m, l, z, a).properties(title = q).interactive()
    sub_charts.append(sub_chart)
    
    

In [326]:
r1 = alt.hconcat(sub_charts[0], sub_charts[1])
r2 = alt.hconcat(sub_charts[2], sub_charts[3])
r3 = alt.hconcat(sub_charts[4], sub_charts[5])

full = alt.vconcat(r1, r2, r3)

In [390]:
full

alt.VConcatChart(...)

<h2>Different Vibe</h2>

In [448]:
preventitive_care['short_question'] = preventitive_care['Question'].replace('Mammogram in the past 2 years among females 50 to 74 years of age', 'Mammograms')
preventitive_care.short_question = preventitive_care.short_question.replace('Up-to-date cervical cancer screening among females 21 to 65 years of age', 'Cervical Cancer Screening')
preventitive_care.short_question = preventitive_care.short_question.replace('Up-to-date colorectal cancer screening among adults 50 to 75 years of age', 'Colorectcal Cancer Screening')
preventitive_care.short_question = preventitive_care.short_question.replace('Routine check-up in the past year among adults 18 years of age or older', 'Routine Check Up')
preventitive_care.short_question = preventitive_care.short_question.replace('Visited a dentist in the past year among adults 18 years of age or older', 'Dentist')
preventitive_care.short_question = preventitive_care.short_question.replace('Had a flu vaccine in the past 12 months among adults 18 years of age or older', 'Flu Vaccine')

In [434]:
preventitive_care['disabled_binary'] = preventitive_care['Stratification1'].apply(lambda s: np.where(s == 'No Disability', 
                                                                                           'No Disability', 'Disabled'))


preventitive_care['margin_of_error'] = preventitive_care['High_Confidence_Limit'] - preventitive_care['Data_Value']
                   

In [485]:
title = alt.TitleParams("Many Disabled People Aren't Up To Date on Important Preventitive Care", 
                         subtitle = '''Age-adjusted prevelance of preventitive care''',
                         anchor = 'start',
                         dx = 45, 
                         fontSize = 22, 
                         subtitleFontSize = 14)

strats = list(preventitive_care.Stratification1.unique())
strats_dropdown = alt.binding_select(options = strats, name = "Disability Type: ")
strats_select = alt.selection_single(fields = ['Stratification1'], init = {'Stratification1' : 'Any Disability'}, bind = strats_dropdown)


l = alt.Chart(preventitive_care, title = title).add_selection(
    strats_select
).mark_rule(
    size = 3
).encode(
    alt.Y('short_question:N', axis = alt.Axis(title = 'Percent Up To Date', labelFontSize = 11, titleFontSize = 13.5)), 
    alt.X('High_Confidence_Limit:Q', scale = alt.Scale(domain = [0.3, 0.9], zero = False), 
          axis = alt.Axis(title = 'Care Type', labelFontSize = 11, titleFontSize = 13.5)), 
    alt.X2('Low_Confidence_Limit:Q'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.2%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.2%')]
).transform_filter(strats_select)

m = alt.Chart(preventitive_care).mark_point(
    filled = True,
    size = 50
).encode(
    y = alt.Y('short_question:N'), 
    x = alt.X('Data_Value:Q'), 
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.2%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.2%')]
).transform_filter(strats_select)

top = alt.layer(l, m
).properties(
    width = 600, 
    height = 300
).interactive()

coords = alt.selection_single(encodings = ['x', 'y'], on = 'mouseover', nearest = True)

base = alt.Chart(mean_by_type).add_selection(coords).mark_rect(
    color = '#fff'
).encode(
    y = alt.Y('short_question:N'),
    x = alt.X('Stratification1:N'), 
    color = alt.condition(coords, alt.Color('short_question:N'), alt.value('lightgray'))
)

text = alt.Chart(mean_by_type).mark_text(
    baseline = 'middle'
).encode(
    y = alt.Y('short_question:N'), 
    x = alt.X('Stratification1:N'),
    text = alt.Text('Data_Value:Q', format=".0%"),
)

#coords = alt.selection_single(encodings = ['x', 'y'], on = 'mouseover', nearest = True)

base = alt.Chart(mean_by_type).mark_rect(
    color = '#fff'
).encode(
    y = alt.Y('short_question:N'),
    x = alt.X('Stratification1:N')
    #, 
    #color = alt.condition(coords, alt.Color('short_question:N'), alt.value('lightgray'))
)

text = alt.Chart(mean_by_type).mark_text(
    baseline = 'middle'
).encode(
    y = alt.Y('short_question:N'), 
    x = alt.X('Stratification1:N'),
    text = alt.Text('Data_Value:Q', format=".0%"),
)


bottom = alt.layer(base, text)
# provider_df = df[df.Question == 'Personal doctor or health care provider among adults 18 years of age or older']
# provider_df = provider_df[provider_df['Year'] == 2020.0]
# bottom_title = alt.TitleParams('Percentage With Primary Care Provider', anchor = 'start', fontSize = 12, dx = 105)

# bottom = alt.Chart(provider_df, title = bottom_title).add_selection(strats_select).mark_bar().encode(
#     y = alt.Y('Stratification1:N', sort = sorting), 
#     x = alt.X('Data_Value:Q'), 
#     opacity = alt.condition(strats_select, alt.value(1.0), alt.value(0.25))
# ).properties(
#     width = 600
# )


alt.vconcat(top, bottom)

alt.VConcatChart(...)

In [486]:
bottom

alt.LayerChart(...)

In [470]:
provider_df = df[df.Question == 'Personal doctor or health care provider among adults 18 years of age or older']
provider_df = provider_df[provider_df['Year'] == 2020.0]

alt.Chart(provider_df).mark_bar().encode(
    y = alt.Y('Stratification1:N'), 
    x = alt.X('Data_Value:Q')
)

alt.Chart(...)

In [369]:
strats = list(preventitive_care.Stratification1.unique())

strats_dropdown = alt.binding_select(options = strats, name = "W")
strats_select = alt.selection_single(fields = ['Stratification1'], bind = strats_dropdown)

lines = alt.Chart(preventitive_care).transform_filter(strats_select).mark_rule(
    #color = 'blue', 
    size = 3
).encode(
    alt.Y('short_question:N', axis = alt.Axis(title = 'Care Type', titleFontSize = 13, labelFontSize = 10.5)), 
    alt.X('High_Confidence_Limit:Q'),
    alt.X2('Low_Confidence_Limit:Q'),
    #color = alt.Color('Question:N', legend = alt.Legend(title = 'Care Type', titleFontSize = 15, labelFontSize = 12)),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
)

mean = alt.Chart(preventitive_care).transform_filter(strats_select).mark_point(
    filled = True,
    size = 50
).encode(
    y = alt.Y('short_question:N', sort = sorting), 
    x = alt.X('Data_Value:Q')
)
  
alt.layer(lines, mean)

alt.LayerChart(...)

In [366]:
strats_dropdown

BindRadioSelect({
  input: 'select',
  name: 'W',
  options: ['Cognitive Disability', 'Hearing Disability', 'Mobility Disability', 'No Disability', 'Vision Disability']
})